MODELO DE RECOMENDAICON DE CANCIONES

In [35]:
import pandas as pd
import numpy as np

In [36]:
music = pd.read_csv("music.csv")

In [37]:
music

,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,Carl Woitschach,158648,1928,1928,0.9950,0.708,0.1950,0.56300,0.1510,-12.428,0.0506,118.469,0.7790,1,10,0,0
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento",Robert Schumann,282133,1928,1928,0.9940,0.379,0.0135,0.90100,0.0763,-28.454,0.0462,83.972,0.0767,1,8,0,0
2,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento",Vladimir Horowitz,282133,1928,1928,0.9940,0.379,0.0135,0.90100,0.0763,-28.454,0.0462,83.972,0.0767,1,8,0,0
3,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,Seweryn Goszczyński,104300,1928,1928,0.6040,0.749,0.2200,0.00000,0.1190,-19.924,0.9290,107.177,0.8800,0,5,0,0
4,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),Francisco Canaro,180760,9/25/28,1928,0.9950,0.781,0.1300,0.88700,0.1110,-14.734,0.0926,108.003,0.7200,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222703,52eycxprLhK3lPcRLbQiVk,How Would I Know,Kygo,180700,5/29/20,2020,0.5380,0.514,0.5390,0.00233,0.1080,-9.332,0.1050,123.700,0.1530,1,7,70,0
222704,52eycxprLhK3lPcRLbQiVk,How Would I Know,Oh Wonder,180700,5/29/20,2020,0.5380,0.514,0.5390,0.00233,0.1080,-9.332,0.1050,123.700,0.1530,1,7,70,0
222705,3wYOGJYD31sLRmBgCvWxa4,I Found You,Cash Cash,167308,2/28/20,2020,0.0714,0.646,0.7610,0.00000,0.2220,-2.557,0.0385,129.916,0.4720,1,1,70,0
222706,3wYOGJYD31sLRmBgCvWxa4,I Found You,Andy Grammer,167308,2/28/20,2020,0.0714,0.646,0.7610,0.00000,0.2220,-2.557,0.0385,129.916,0.4720,1,1,70,0


In [38]:
spotify_data = music.copy()

CREACION DEL MODELO

1. Modelo de Nearest Neighbors

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [40]:
# Segmento las columnas relevantes
columnas_relevantes = ['year','acousticness', 'danceability', 'energy', 'loudness','valence', 'popularity']

X = spotify_data[columnas_relevantes]


In [41]:
# Normalizar las características (opcional, pero recomendado)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [42]:
# Separamos el conjunto de  datos
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

In [67]:
# Entrenar el modelo usando el algoritmo de Nearest Neighbors
nn = NearestNeighbors(n_neighbors=5, algorithm='auto') # auto - ball_tree - kd_tree - brute
nn.fit(X_train)

NearestNeighbors()

In [68]:
# Paso 4: Hacer recomendaciones
def recommend_songs(song_index, n_recommendations=5):
    distances, indices = nn.kneighbors([X_test[song_index]], n_neighbors=n_recommendations+1)
    return indices[0][1:]  # Excluyendo el primer elemento que es la canción en sí misma

In [69]:
# Ejemplo: Recomendar canciones para la primera canción en el conjunto de prueba
recommended_indices = recommend_songs(100)
recommended_songs = spotify_data.iloc[recommended_indices]
print("Canciones recomendadas:")
print(recommended_songs[['name', 'artists']])

Canciones recomendadas:
                      name               artists
162683              Nobody              Skindred
149058     We Can Get Down  A Tribe Called Quest
137477         Estaba Solo        Los Temerarios
80014   Ku'u Lei, Ku'u Ipo                  Hapa
115413               GOOBA               6ix9ine


In [70]:
df = spotify_data

In [71]:
# Evaluar el modelo

# Función para calcular la precisión
def precision_at_k(k, n_samples=100):
    relevant_recommendations = 0
    for i in range(n_samples):
        recommendations = recommend_songs(i, n_recommendations=k)
        for rec in recommendations:
            if df.iloc[rec]['popularity'] > df['popularity'].quantile(0.75):  # Consideramos relevantes las canciones más populares
                relevant_recommendations += 1
    return relevant_recommendations / (n_samples * k)

# Función para calcular la cobertura
def coverage_at_k(k, n_samples=100):
    all_recommendations = set()
    for i in range(n_samples):
        recommendations = recommend_songs(i, n_recommendations=k)
        all_recommendations.update(recommendations)
    return len(all_recommendations) / len(df)

# Función para calcular la diversidad
def diversity_at_k(k, n_samples=100):
    total_diversity = 0
    for i in range(n_samples):
        recommendations = recommend_songs(i, n_recommendations=k)
        pairwise_distances = []
        for j in range(len(recommendations)):
            for l in range(j + 1, len(recommendations)):
                pairwise_distances.append(np.linalg.norm(X_scaled[recommendations[j]] - X_scaled[recommendations[l]]))
        if pairwise_distances:
            total_diversity += np.mean(pairwise_distances)
    return total_diversity / n_samples

# Función para calcular la novedad
def novelty_at_k(k, n_samples=100):
    total_novelty = 0
    for i in range(n_samples):
        recommendations = recommend_songs(i, n_recommendations=k)
        for rec in recommendations:
            total_novelty += 1 - (df.iloc[rec]['popularity'] / df['popularity'].max())  # Asumimos que las canciones menos populares son más novedosas
    return total_novelty / (n_samples * k)

# Evaluar el modelo
k = 5
n_samples = 100

precision = precision_at_k(k, n_samples)
coverage = coverage_at_k(k, n_samples)
diversity = diversity_at_k(k, n_samples)
novelty = novelty_at_k(k, n_samples)

print(f"Precisión a {k}: {precision}")
print(f"Cobertura a {k}: {coverage}")
print(f"Diversidad a {k}: {diversity}")
print(f"Novedad a {k}: {novelty}")


Precisión a 5: 0.236
Cobertura a 5: 0.002245092228388742
Diversidad a 5: 3.553962116045526
Novedad a 5: 0.7282600000000002


Precisión a 5: 0.236

Esto significa que, en promedio, el 23.6% de las canciones recomendadas están entre las más populares (en el cuartil superior de popularidad). Es una medida de qué tan bien el modelo recomienda canciones que probablemente le gusten al usuario.

Cobertura a 5: 0.0022

La cobertura es muy baja, indica que solo el 0.22% de todas las canciones del dataset son recomendadas. Esto sugiere que el modelo puede no estar recomendando una amplia variedad de canciones.

Diversidad a 5: 3.55

La diversidad es relativamente baja, indicando que las canciones recomendadas tienden a ser similares entre sí en el espacio de características. Valores más altos serían deseables para una mayor diversidad.

Novedad a 5: 0.728

La novedad es alta, lo cual es bueno. Indica que las canciones recomendadas tienden a ser menos populares, lo que sugiere que el modelo está recomendando canciones menos conocidas.<br>


Interpretación:
La precisión muestra que el modelo recomienda canciones populares entre las recomendaciones, pero la cobertura es muy baja, lo que significa que no abarca muchas canciones diferentes.
La diversidad es moderada, lo que sugiere que las recomendaciones tienden a ser variadas pero aún podrían mejorarse.
La alta novedad indica que el modelo recomienda canciones menos populares, lo cual puede ser positivo si el objetivo es recomendar nuevas canciones al usuario.
Basándonos en estos resultados, podrías intentar ajustar el modelo para mejorar la cobertura y la diversidad, quizás explorando diferentes valores de n_neighbors en NearestNeighbors o ajustando otros hiperparámetros para obtener recomendaciones más diversas y representativas.






